## 🦜🔗 LangChain RAG Agent (RAG nur bei Bedarf)

In [ ]:
#%pip install -r requirements.txt

In [ ]:
from langchain_community.vectorstores.faiss import FAISS
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from helpers import llm, embeddings
from dotenv import load_dotenv
load_dotenv()

#### Wir erstellen eine kleine in-memory FAISS Datenbank.

Zu Demo-Zwecken speichern wir sie kurz noch ab, dann kann man sie jederzeit wieder neu laden

In [ ]:
loader = PyPDFLoader("LangChain.pdf")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
docs = text_splitter.split_documents(documents)
db = FAISS.from_documents(docs, embeddings())
db.save_local("./vectorstore")

#### Wir benutzen LangChain-Magie um aus der Vektor-Datenbank einen Retriever und aus dem Retriever ein Tool zu machen, das der Agent benutzen kann

In [ ]:
db = FAISS.load_local("./vectorstore", embeddings(), allow_dangerous_deserialization=True)
retriever = db.as_retriever()
tool = create_retriever_tool(
    retriever,
    "search_langchain_documentation",
    "This is the best place to look for any information about LangChain."
)
tools = [tool]

#### Wir bauen den Agenten (Noch mehr Magie) und geben ihm das Tool.

In [ ]:
from langgraph.prebuilt import create_agent_executor
from langchain.agents import create_openai_functions_agent
from langchain import hub
prompt = hub.pull("reactagent/rag")
agent_runnable = create_openai_functions_agent(llm(), tools, prompt)
agent_executor = create_agent_executor(agent_runnable, tools)

#### Wir definieren eine kleine Funktion, die die Ausgabe formatiert.

Diese Arbeit muss man üblicherweise selbst tun, weil LangChain ja nicht weiß, welches Format man am Ende braucht.

In [ ]:

def formatted_output_writer(data):
    return data.get("agent_outcome").return_values.get("output")

app = agent_executor | formatted_output_writer

#### Lets go!

In [ ]:
inputs = {"input": "How do I use LangChain with a Vectorstore?"}
output = app.invoke(inputs)
output = app.invoke(inputs)
print(f"Agent result: {output}")